In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from fastai.tabular.all import * 
from mock import Mock
from tabnet.utils import *
from tabnet.model import *

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [4]:
class LinDecoder(Module):
    def __init__(self, n_cat, n_cont, n_d, ps=0.1, **kwargs):
        store_attr()
        
        self.decoder = nn.Sequential(
            LinBnDrop(n_d, 256, p=ps, act=Mish()),
            LinBnDrop(256, 512, p=ps, act=Mish()),
            LinBnDrop(512, 1024, p=ps, act=Mish())
        )
        
        self.decoder_cont = nn.Sequential(
            LinBnDrop(1024, n_cont, p=ps, bn=False, act=None),
        )
        
        self.decoder_cat = LinBnDrop(1024, n_cat, p=ps, bn=False, act=None)

        
    def forward(self, x):
        xs = x.sum(dim=1)
        decoded = self.decoder(xs)
        decoded_cats = self.decoder_cat(decoded)
        decoded_conts = self.decoder_cont(decoded)

        return torch.cat([decoded_cats, decoded_conts], dim=1)

In [5]:
def _create_shared_blocks(n_in, n_out, n_shared):
    return [_initial_block(n_in, n_out)] + \
            [_rest_block(n_out) for _ in range(n_shared-1)]

def _initial_block(n_in, n_out):
    return nn.Linear(n_in, 2*n_out, bias=False)

def _rest_block(n):
    return nn.Linear(n, 2*n, bias=False)


In [6]:
class TabNetDec2(TabNetBase):
    def __init__(self, n_cat, n_cont, n_d, n_a, n_shared_ft_blocks, n_dec_steps, **kwargs):
        store_attr()
        super().__init__(n_d=n_d, n_a=n_a, n_shared_ft_blocks=n_shared_ft_blocks, **kwargs)
        
        shared_ft_blocks = _create_shared_blocks(self.n_d, self.n_d + self.n_a, self.n_shared_ft_blocks)
        
        self.steps = nn.ModuleList([
                            nn.Sequential(
                                self._create_feature_transform(shared_ft_blocks),
                                nn.Linear(self.n_d+self.n_a, n_cat+n_cont)) for _ in range(self.n_dec_steps)
                        ])
        
        
        
        
    def forward(self, x):
        xs = x.sum(dim=1)
        
        output = 0 
        
        for step in self.steps:
            output = output + step(xs)
        
        return output

In [7]:
class MRL1(Module):
    def __init__(self, lambda_reg=1e-4, eps=1e-5): store_attr()
    
    def forward(self, preds, targ):
        preds, targ = preds*(1-self.S), targ*(1-self.S)
        norm = (targ - targ.mean(dim=0)).pow(2).sum(dim=0).sqrt()
        norm_mask = norm >= 1e-6
        norm = norm[norm_mask]
        error = (preds - targ)
        error = error[:,norm_mask]

        loss = (error / norm).pow(2).sum(dim=1).mean()
        return loss

In [8]:
class MRL2(Module):
    def __init__(self, lambda_reg=1e-4, eps=1e-5): store_attr()
    
    def forward(self, preds, targ):
        preds, targ = preds*(1-self.S), targ*(1-self.S)
        norm = (targ - targ.mean(dim=0)).pow(2).sum(dim=0).sqrt()
        error = (preds - targ)

        loss = (error / norm).abs().sum(dim=1).mean()
        return loss

# Tests

### SS Forest

In [9]:
data_dir = Path('./data')

In [10]:
def extract_gzip(file, dest=None):
    import gzip
    dest = dest or Path(dest)
    with gzip.open(file, 'rb') as f_in:
        with open(dest / file.stem, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [11]:
forest_type_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'
forest_path = untar_data(forest_type_url, dest=data_dir, extract_func=extract_gzip)

In [12]:
target = "Covertype"

cat_names = [
    "Wilderness_Area1", "Wilderness_Area2", "Wilderness_Area3",
    "Wilderness_Area4", "Soil_Type1", "Soil_Type2", "Soil_Type3", "Soil_Type4",
    "Soil_Type5", "Soil_Type6", "Soil_Type7", "Soil_Type8", "Soil_Type9",
    "Soil_Type10", "Soil_Type11", "Soil_Type12", "Soil_Type13", "Soil_Type14",
    "Soil_Type15", "Soil_Type16", "Soil_Type17", "Soil_Type18", "Soil_Type19",
    "Soil_Type20", "Soil_Type21", "Soil_Type22", "Soil_Type23", "Soil_Type24",
    "Soil_Type25", "Soil_Type26", "Soil_Type27", "Soil_Type28", "Soil_Type29",
    "Soil_Type30", "Soil_Type31", "Soil_Type32", "Soil_Type33", "Soil_Type34",
    "Soil_Type35", "Soil_Type36", "Soil_Type37", "Soil_Type38", "Soil_Type39",
    "Soil_Type40"
]

cont_names = [
    "Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
    "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points"
]

feature_columns = (
    cont_names + cat_names + [target])

params = dict(cont_names = cont_names, y_names = target, cat_names = cat_names)
procs=[Categorify, FillMissing, Normalize]
model_params = dict(n_d=64, n_a=64, n_steps=5, virtual_batch_size=512, gamma=1.5, bs=1024*16,
                    lambda_sparse=1e-4, momentum=0.7, n_shared_ft_blocks=2, n_independent_ft_blocks=2,
                    n_dec_steps=10, p=0.8, curriculum=True)

In [13]:
df = pd.read_csv(forest_path, header=None, names=feature_columns).sample(n=200_000)
df.shape

(200000, 55)

In [14]:
val_pct = 0.2

In [15]:
loss_func = MRL2()

#head = lambda n_cat, n_cont, **kwargs: LinDecoder(n_cat, n_cont, **kwargs)
head = lambda n_cat, n_cont, **kwargs: TabNetDec2(n_cat, n_cont, **kwargs)

In [18]:
vals = []
for val_pct in sorted([0.2, 0.4, 0.6]*2):
    print(val_pct)
    (before, after) = score_before_after_ss(df, params, val_pct=val_pct, decoder_head=head, 
                        loss_func=loss_func, cycle_lr=[(350, slice(5e-3, 1e-1)), (100, slice(1e-3, 1e-1/2)),
                                                       (350, slice(5e-3, 1e-1))], **model_params)
    vals.append((before.item(), after.item(), val_pct))
    res = pd.DataFrame(vals, columns=['before','after','val'])
    res.to_csv('forest_res.csv')

0.2
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8, 'curriculum': True}


epoch,train_loss,valid_loss,accuracy,time
0,2.925017,2.380807,0.482475,00:03
1,1.951404,1.109250,0.602825,00:03
2,1.526027,0.902281,0.663025,00:03
3,1.294128,0.826152,0.672075,00:03
4,1.142889,0.803640,0.673050,00:03
5,1.038442,0.780601,0.678600,00:03
6,0.962587,0.747324,0.685675,00:03
7,0.905106,0.726009,0.693925,00:03
8,0.860777,0.702040,0.694225,00:03
9,0.828547,0.711876,0.697775,00:03


epoch,train_loss,valid_loss,mse,time
0,0.670779,0.201447,0.246830,00:07
1,0.449109,0.169819,0.230805,00:07
2,0.343572,0.135362,0.245165,00:07
3,0.284154,0.129907,0.245388,00:07
4,0.246157,0.139271,0.249408,00:07
5,0.219997,0.154999,0.361067,00:07
6,0.200900,0.128274,0.239571,00:07
7,0.186258,0.160028,0.557504,00:07
8,0.176567,0.196912,1.811295,00:07
9,0.170934,0.159659,0.596846,00:07


epoch,train_loss,valid_loss,accuracy,time
0,2.549387,1.760486,0.501350,00:03
1,1.810809,1.657297,0.506350,00:03
2,1.505343,1.268475,0.522725,00:03
3,1.326320,2.158410,0.498225,00:03
4,1.203488,1.933436,0.437775,00:03
5,1.111365,1.676360,0.489700,00:03
6,1.039941,1.296988,0.580950,00:03
7,0.981495,1.802179,0.582375,00:03
8,0.935443,1.157109,0.604600,00:03
9,0.897912,0.996482,0.627625,00:03


0.2
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8, 'curriculum': True}


...

epoch,train_loss,valid_loss,accuracy,time
0,3.687016,2.711525,0.457708,00:01
1,3.113081,1.966787,0.465975,00:01
2,2.622377,1.511139,0.498858,00:01
3,2.259630,1.705456,0.482083,00:01
4,2.004467,1.220046,0.527158,00:01
5,1.823889,1.338458,0.480600,00:01
6,1.687479,1.542360,0.437525,00:01
7,1.579086,1.670842,0.419183,00:01
8,1.490310,1.588423,0.406733,00:01
9,1.415691,1.609312,0.408925,00:01


0.6
{'n_d': 64, 'n_a': 64, 'n_steps': 5, 'virtual_batch_size': 512, 'gamma': 1.5, 'bs': 16384, 'lambda_sparse': 0.0001, 'momentum': 0.7, 'n_shared_ft_blocks': 2, 'n_independent_ft_blocks': 2, 'n_dec_steps': 10, 'p': 0.8, 'curriculum': True}


epoch,train_loss,valid_loss,accuracy,time
0,3.224725,2.552136,0.417175,00:01
1,2.515308,1.888419,0.495967,00:01
2,2.041324,1.558810,0.488658,00:01
3,1.766384,1.258079,0.492692,00:01
4,1.579731,1.257125,0.482250,00:01
5,1.447387,1.165910,0.505200,00:01
6,1.348395,1.042867,0.549200,00:01
7,1.266287,1.018165,0.554592,00:01
8,1.198230,0.984390,0.551233,00:01
9,1.141161,0.890430,0.618983,00:01


epoch,train_loss,valid_loss,mse,time
0,0.663546,0.334052,0.275109,00:07
1,0.442413,0.124994,0.227834,00:07
2,0.340965,0.122547,0.229913,00:07
3,0.283443,0.123205,0.231778,00:07
4,0.246833,0.139525,0.230600,00:07
5,0.221015,0.117477,0.229261,00:07
6,0.201861,0.144614,0.368460,00:07
7,0.187330,0.150761,0.600340,00:07
8,0.177517,0.158123,0.428454,00:07
9,0.172187,0.173182,0.765062,00:07


epoch,train_loss,valid_loss,accuracy,time
0,3.817264,2.825176,0.296458,00:02
1,3.271517,2.249825,0.431217,00:02
2,2.762721,1.658904,0.463750,00:02
3,2.353474,2.024934,0.481075,00:02
4,2.069092,1.925478,0.377342,00:02
5,1.874271,2.594393,0.393733,00:02
6,1.734284,1.743225,0.509450,00:02
7,1.625026,2.019701,0.469758,00:02
8,1.537776,4.944553,0.455342,00:02
9,1.466219,4.118552,0.492033,00:02


# Export

In [ ]:
from nbdev.export import notebook2script
notebook2script()